# NeuroMANCER: System ID Tutorial

This notebook outlines the basic structure of a training script for system identification with deep state space models.

We begin by importing the necessary components from NeuroMANCER, as well as specifying some hyperparameters used to define the structure and optimization of our model:

In [19]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import slim
import psl

from neuromancer import blocks, estimators, dynamics, arg
from neuromancer.activations import activations
from neuromancer.visuals import VisualizerOpen
from neuromancer.trainer import Trainer
from neuromancer.problem import Problem
from neuromancer.constraint import Objective
from neuromancer.simulators import OpenLoopSimulator, MultiSequenceOpenLoopSimulator
from neuromancer.callbacks import SysIDCallback
from neuromancer.loggers import BasicLogger, MLFlowLogger
from neuromancer.dataset import read_file, normalize_data, split_sequence_data, SequenceDataset
from neuromancer.constraint import Variable


# params for configuration
params = {
    "nsteps": 32,             # Prediction horizon length.
    "nsim": 10000,            # Number of temporal datapoints in the simulated dataset.
    "data_seed": 408,         # Random seed for the dataset emulator.
    "norm": ["U", "D", "Y"],  # Norms imposed on raw variables. U: inputs, D: disturbances, Y: outputs.
    "Q_y": 1.0,               # Output tracking penalty weight.
    "Q_e": 1.0,               # State estimator hidden prediction penalty weight.
    "Q_sub": 1.0,             # Linear maps regularization weight.
    "Q_con_x": 1.0,           # Hidden state constraints penalty weight.
    "Q_dx": 0.1,              # Penalty weight on hidden state difference in one time step. 
    "savedir": "test",        # Where should your trained model and plots be saved (temp).
    "eval_metric": "loop_dev_ref_loss",   # Metric for model selection and early stopping.
    "metrics": ["nstep_dev_loss", "loop_dev_loss", "best_loop_dev_loss",
               "nstep_dev_ref_loss", "loop_dev_ref_loss"],   # Metrics to be logged.
    "epochs": 300,            # Number of training epochs.
    "lr": 0.001,              # Learning rate.
    "patience": 100,          # Early stopping patience.
    "warmup": 10,             # Early stopping warmup.
}
device = "cuda:0" if torch.cuda.is_available() else "cpu"

/home/tuor369/anaconda3/envs/neuromancer/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Datasets
NeuroMANCER currently supports both static and emulated datasets (generated from governing equations for various systems) via [PSL](https://github.com/pnnl/psl). Here, we define get_sequence_dataloaders() function for generating dataloader from the raw  data given as dictionary.  
For more details see datasets tutorial.

In [20]:
def get_sequence_dataloaders(
    data, nsteps, moving_horizon=False, norm_type="zero-one", split_ratio=None, num_workers=0,
):
    """This will generate dataloaders and open-loop sequence dictionaries for a given dictionary of
    data. Dataloaders are hard-coded for full-batch training to match NeuroMANCER's original
    training setup.

    :param data: (dict str: np.array or list[dict str: np.array]) data dictionary or list of data
        dictionaries; if latter is provided, multi-sequence datasets are created and splits are
        computed over the number of sequences rather than their lengths.
    :param nsteps: (int) length of windowed subsequences for N-step training.
    :param moving_horizon: (bool) whether to use moving horizon batching.
    :param norm_type: (str) type of normalization; see function `normalize_data` for more info.
    :param split_ratio: (list float) percentage of data in train and development splits; see
        function `split_sequence_data` for more info.
    """

    data, _ = normalize_data(data, norm_type)
    train_data, dev_data, test_data = split_sequence_data(data, nsteps, moving_horizon, split_ratio)

    train_data = SequenceDataset(
        train_data,
        nsteps=nsteps,
        moving_horizon=moving_horizon,
        name="train",
    )
    dev_data = SequenceDataset(
        dev_data,
        nsteps=nsteps,
        moving_horizon=moving_horizon,
        name="dev",
    )
    test_data = SequenceDataset(
        test_data,
        nsteps=nsteps,
        moving_horizon=moving_horizon,
        name="test",
    )

    train_loop = train_data.get_full_sequence()
    dev_loop = dev_data.get_full_sequence()
    test_loop = test_data.get_full_sequence()

    train_data = DataLoader(
        train_data,
        batch_size=len(train_data),
        shuffle=False,
        collate_fn=train_data.collate_fn,
        num_workers=num_workers,
    )
    dev_data = DataLoader(
        dev_data,
        batch_size=len(dev_data),
        shuffle=False,
        collate_fn=dev_data.collate_fn,
        num_workers=num_workers,
    )
    test_data = DataLoader(
        test_data,
        batch_size=len(test_data),
        shuffle=False,
        collate_fn=test_data.collate_fn,
        num_workers=num_workers,
    )

    return (train_data, dev_data, test_data), (train_loop, dev_loop, test_loop), train_data.dataset.dims


In [21]:
# load dictionary dataset with raw data
# for available systems in PSL library check: psl.systems.keys()
# for available datasets in PSL library check: psl.datasets.keys()

system = 'aero'         # keyword of selected system

#  load raw dataset
if system in psl.emulators:
    data = psl.emulators[system](nsim=nsim, ninit=0, seed=args.data_seed).simulate()
elif system in psl.datasets:
    data = read_file(psl.datasets[system])
else:
    data = read_file(system)

Processed dataset format:  
nsim = number of time steps in the dataset time series  
nsteps = legth of the prediction horizon  
Y = observed outputs  
U = inputs  
D = disturbances  
Yp = past trajectories generated as Y[0:-nsteps]  
Yf = future trajectories generated as Y[nesteps:]  
data format in dataset dictionaries: train_data['key']: torch.Size([batch size (prediction horizon), number of batches, number of variables])

In [22]:
#  Train, Development, Test sets - nstep and loop format
nstep_data, loop_data, dims = get_sequence_dataloaders(data, params['nsteps'])
train_data, dev_data, test_data = nstep_data
train_loop, dev_loop, test_loop = loop_data

# input, output dimensions
nu = dims['U'][1]
ny = dims['Y'][1]

## State Estimator

To generate initial states during $N$-step training, we make use of state estimators which use the past outputs of the system to estimate the next state variable. NeuroMANCER provides a host of parametric state estimators for this purpose; in this case, we'll use a fully connected neural network. Each state estimator in neuromancer.estimators represents more generic component class representing arbitrary nn.Module with input_keys and output_keys for mapping neuromancer variabels.  
For more details see components and constraints tutorials.

In [23]:
activation = activations['relu']
linmap = slim.maps['linear']

nx = 90  # size of the latent variables
estimator = estimators.MLPEstimator(
    {**dims, "x0": (nx,)},
    nsteps=params['nsteps'],  # future window Nf
    window_size=params['nsteps'],  # past window Np <= Nf
    bias=True,
    linear_map=linmap,
    nonlin=activation,
    hsizes=[90, 120, 90],
    input_keys=["Yp"],
    linargs={},
    name='estimator',
)

# input output keys of the estimator 
print(estimator.input_keys)
print(estimator.output_keys)

# dynamics_model mapping of input keys to output keys
print(estimator)
# x0 = estimator(Yp)
# x0: initial values of latent variables estimated from time lagged outputs Yp

['Yp']
['x0_estimator', 'reg_error_estimator']
estimator(Yp) -> x0_estimator, reg_error_estimator


/home/tuor369/anaconda3/envs/neuromancer/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## State Space Model

Next, we define our block-structured state space model (SSM) as neuromancer component. These models divide system dynamics into separate functional components to decouple the behaviors of inputs, state transitions, and outputs. Using the `BlockSSM` constructor, we can build SSMs with arbitrary components, each with their own nonlinearities and structured weight parameterizations.  
For more details on how to construct arbitrary acyclic computational graphs with see components tutorial.

In [24]:
# define blocks
fx = blocks.RNN(nx, nx, linear_map=linmap,
                nonlin=activations['softexp'], hsizes=[60, 60])
linargs = {"sigma_min": 0.5, "sigma_max": 1.0}
fy = slim.maps['softSVD'](nx, ny, linargs=linargs)
fu = blocks.MLP(nu, nx, hsizes=[60, 60]) if nu != 0 else None

# define state space model
dynamics_model = dynamics.BlockSSM(fx, fy, fu=fu, name='dynamics', xou=torch.add,
                                   input_key_map={"x0": f"x0_{estimator.name}"})

# input output keys of the SSM 
print(dynamics_model.input_keys)
print(dynamics_model.output_keys)

# dynamics_model mapping of input keys to output keys
print(dynamics_model)
# Yf = dynamics_model(x0, Uf)
# Uf: future control actions
# Yf: predicted outputs

['Uf', 'Yf', 'x0_estimator']
['fU_dynamics', 'X_pred_dynamics', 'Y_pred_dynamics', 'reg_error_dynamics']
dynamics(Uf, Yf, x0_estimator) -> fU_dynamics, X_pred_dynamics, Y_pred_dynamics, reg_error_dynamics


## Neuromancer Variables
Here we leverage Neuromancer's variable abstraction for intuitive high-level definition of objective temrs and constraints. Each variable has assigned uniqye name that either exists as a key in the dataset dictionary (e.g., "Yf"), or is listed as output key in one of our components.

In [25]:
# bounds
xmin = -0.2
xmax = 1.2
dxudmin = -0.5
dxudmax = 0.5

# loss terms and constraints definition via variable class: neuromancer variable declaration
yhat = Variable(f"Y_pred_{dynamics_model.name}")
y = Variable("Yf")
x0 = Variable(f"x0_{estimator.name}")
xhat = Variable(f"X_pred_{dynamics_model.name}")
est_reg = Variable(f"reg_error_{estimator.name}")
dyn_reg = Variable(f"reg_error_{dynamics_model.name}")

## Objective Terms
To optimize the weights of our model, we first use mean-squared error as our objective to minimize the error between ground-truth observables and those predicted by our SSM. We additionally impose a loss on the estimator that ensures its state predictions align with ground-truth state observations. Further we add state smoothening penalty, and regularizations terms imposed on the model weights.  

In addition to minimizing the mean-squared error of predicted and expected observables, we may also want to impose further constraints on different model components to enforce certain model behaviors. Our first constraint will ensure the smoothness of state transitions by minimizing the mean-squared error between consecutive state predictions. Our second and third constraints impose lower and upper bounds on output observations, specifically the interval $[-0.2, 1.2]$.

For more details see constraints tutorial.

In [26]:
# define loss function terms
reference_loss = params['Q_y']*((yhat == y)^2)
estimator_loss = params['Q_e']*((x0[1:] == xhat[-1, :-1, :])^2)
state_smoothing = params['Q_dx']*((xhat[1:] == xhat[:-1])^2)
regularization = params['Q_sub']*((est_reg + dyn_reg == 0)^2)
# define constraints
observation_lower_bound_penalty = params['Q_con_x']*(yhat > xmin)
observation_upper_bound_penalty = params['Q_con_x']*(yhat < xmax)
# custom loss and constraints names used as metrics
reference_loss.name = "ref_loss"
estimator_loss.name = "arrival_cost"
regularization.name = "reg_error"
observation_lower_bound_penalty.name = "y_low_bound_error"
observation_upper_bound_penalty.name = "y_up_bound_error"

## Putting It All Together

Having defined our model components, objective terms, and constraints, we can now combine each into an optimization problem using NeuroMANCER's `Problem` class. This will produce our final model, connecting all of the underlying components necessary to process our data, generate predictions, and optimize the model components.

In [27]:
# list of objectives and constraints
objectives = [regularization, reference_loss, estimator_loss]
constraints = [
    state_smoothing,
    observation_lower_bound_penalty,
    observation_upper_bound_penalty,
]
# list of components
components = [estimator, dynamics_model]
# constrained optimization problem = objectives + constraints + components
model = Problem(objectives, constraints, components)
model = model.to(device)

## Training Our Model

For training models, NeuroMANCER provides a `Trainer` class which encapsulates all of the training and evaluation logic necessary for $N$-step training, and additionally provides options for experiment logging, visualization, and roll-out simulation in the loop. To get started, we instantiate PyTorch's AdamW optimizer with our model's parameters. We then provide some components which are specific to the NeuroMANCER training regime:

* The `OpenLoopSimulator` is used to simulate a full roll-out of the SSM starting from a single initial state. Unlike the $N$-step simulation used during training, this is meant to evaluate the performance of the model in a free-running manner rather than over short windows of time.
* The `BasicLogger` is used to output user-specified metrics during training and evaluation. It also handles saving model checkpoints.
* The `VisualizerOpen` class generates plots showing predicted vs. ground-truth outputs for both $N$-step and open-loop model simulations, providing a useful visual assessment of model performance.

With all of these components defined, we can then instantiate our `Trainer` class. We provide the model and dataset, the optimizer, and the aforementioned helper components. We additionally specify the number of epochs, an evaluation metric used to determine the best-performing model, and a patience value; this latter value determines how many epochs should be completed without any improvement in the evaluation metric before discontinuing training in order to avoid overfitting and wasted computation.

In [28]:
# define callback
simulator = OpenLoopSimulator(
    model, train_loop, dev_loop, test_loop, eval_sim=True, device=device,
) if isinstance(train_loop, dict) else MultiSequenceOpenLoopSimulator(
    model, train_loop, dev_loop, test_loop, eval_sim=True, device=device,
)
visualizer = VisualizerOpen(
    dynamics_model,
    1,
    params['savedir'],
    training_visuals=False,
    trace_movie=False,
)
callback = SysIDCallback(simulator, visualizer)
# define logger
logger = BasicLogger(args=None, savedir= params['savedir'], verbosity=1, stdout=params['metrics'])
# select optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=params['lr'])

# define trainer
trainer = Trainer(
    model,
    train_data,
    dev_data,
    test_data,
    optimizer,
    callback=callback,
    logger=logger,
    epochs=params['epochs'],
    eval_metric=params['eval_metric'],
    patience=params['patience'],
    warmup=params['warmup'],
    device=device,
)

None
Number of parameters: 85022


Now we enter the training loop by calling the `train()` method on the `trainer` object; upon training completion, this method will return the best-observed model according to the evaluation metric specified previously.

In [29]:
best_model = trainer.train()

/home/tuor369/anaconda3/envs/neuromancer/lib/python3.8/site-packages/torch/nn/modules/container.py:435: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")


epoch: 0	nstep_dev_ref_loss: 0.31286	nstep_dev_loss: 0.45510	loop_dev_ref_loss: 0.31246	loop_dev_loss: nan	eltime:  4.10042
epoch: 1	nstep_dev_ref_loss: 0.28439	nstep_dev_loss: 0.33467	loop_dev_ref_loss: 0.28394	loop_dev_loss: nan	eltime:  4.26508
epoch: 2	nstep_dev_ref_loss: 0.25881	nstep_dev_loss: 0.32463	loop_dev_ref_loss: 0.25832	loop_dev_loss: nan	eltime:  4.41580
epoch: 3	nstep_dev_ref_loss: 0.23479	nstep_dev_loss: 0.33127	loop_dev_ref_loss: 0.23428	loop_dev_loss: nan	eltime:  4.57513
epoch: 4	nstep_dev_ref_loss: 0.21251	nstep_dev_loss: 0.30156	loop_dev_ref_loss: 0.21199	loop_dev_loss: nan	eltime:  4.72994
epoch: 5	nstep_dev_ref_loss: 0.19302	nstep_dev_loss: 0.26531	loop_dev_ref_loss: 0.19253	loop_dev_loss: nan	eltime:  4.87832
epoch: 6	nstep_dev_ref_loss: 0.17780	nstep_dev_loss: 0.24905	loop_dev_ref_loss: 0.17738	loop_dev_loss: nan	eltime:  5.02881
epoch: 7	nstep_dev_ref_loss: 0.16820	nstep_dev_loss: 0.25671	loop_dev_ref_loss: 0.16793	loop_dev_loss: nan	eltime:  5.16455
epoch: 8

epoch: 66	nstep_dev_ref_loss: 0.09878	nstep_dev_loss: 0.13736	loop_dev_ref_loss: 0.09801	loop_dev_loss: nan	eltime:  13.82958
epoch: 67	nstep_dev_ref_loss: 0.09866	nstep_dev_loss: 0.13701	loop_dev_ref_loss: 0.09790	loop_dev_loss: nan	eltime:  13.98176
epoch: 68	nstep_dev_ref_loss: 0.09843	nstep_dev_loss: 0.13671	loop_dev_ref_loss: 0.09769	loop_dev_loss: nan	eltime:  14.14168
epoch: 69	nstep_dev_ref_loss: 0.09803	nstep_dev_loss: 0.13628	loop_dev_ref_loss: 0.09729	loop_dev_loss: nan	eltime:  14.32669
epoch: 70	nstep_dev_ref_loss: 0.09746	nstep_dev_loss: 0.13566	loop_dev_ref_loss: 0.09673	loop_dev_loss: nan	eltime:  14.48147
epoch: 71	nstep_dev_ref_loss: 0.09680	nstep_dev_loss: 0.13489	loop_dev_ref_loss: 0.09606	loop_dev_loss: nan	eltime:  14.62964
epoch: 72	nstep_dev_ref_loss: 0.09612	nstep_dev_loss: 0.13411	loop_dev_ref_loss: 0.09538	loop_dev_loss: nan	eltime:  14.77485
epoch: 73	nstep_dev_ref_loss: 0.09547	nstep_dev_loss: 0.13345	loop_dev_ref_loss: 0.09474	loop_dev_loss: nan	eltime:  1

epoch: 132	nstep_dev_ref_loss: 0.06751	nstep_dev_loss: 0.11924	loop_dev_ref_loss: 0.06743	loop_dev_loss: nan	eltime:  23.82468
epoch: 133	nstep_dev_ref_loss: 0.06711	nstep_dev_loss: 0.11877	loop_dev_ref_loss: 0.06702	loop_dev_loss: nan	eltime:  23.97076
epoch: 134	nstep_dev_ref_loss: 0.06668	nstep_dev_loss: 0.11819	loop_dev_ref_loss: 0.06658	loop_dev_loss: nan	eltime:  24.11173
epoch: 135	nstep_dev_ref_loss: 0.06627	nstep_dev_loss: 0.11761	loop_dev_ref_loss: 0.06616	loop_dev_loss: nan	eltime:  24.27135
epoch: 136	nstep_dev_ref_loss: 0.06590	nstep_dev_loss: 0.11715	loop_dev_ref_loss: 0.06578	loop_dev_loss: nan	eltime:  24.42984
epoch: 137	nstep_dev_ref_loss: 0.06557	nstep_dev_loss: 0.11676	loop_dev_ref_loss: 0.06544	loop_dev_loss: nan	eltime:  24.58096
epoch: 138	nstep_dev_ref_loss: 0.06524	nstep_dev_loss: 0.11637	loop_dev_ref_loss: 0.06510	loop_dev_loss: nan	eltime:  24.74063
epoch: 139	nstep_dev_ref_loss: 0.06489	nstep_dev_loss: 0.11591	loop_dev_ref_loss: 0.06474	loop_dev_loss: nan	el

epoch: 198	nstep_dev_ref_loss: 0.04600	nstep_dev_loss: 0.09496	loop_dev_ref_loss: 0.04537	loop_dev_loss: nan	eltime:  33.67400
epoch: 199	nstep_dev_ref_loss: 0.04557	nstep_dev_loss: 0.09418	loop_dev_ref_loss: 0.04491	loop_dev_loss: nan	eltime:  33.82111
epoch: 200	nstep_dev_ref_loss: 0.04548	nstep_dev_loss: 0.09489	loop_dev_ref_loss: 0.04484	loop_dev_loss: nan	eltime:  33.96721
epoch: 201	nstep_dev_ref_loss: 0.04491	nstep_dev_loss: 0.09350	loop_dev_ref_loss: 0.04422	loop_dev_loss: nan	eltime:  34.11034
epoch: 202	nstep_dev_ref_loss: 0.04502	nstep_dev_loss: 0.09522	loop_dev_ref_loss: 0.04439	loop_dev_loss: nan	eltime:  34.25365
epoch: 203	nstep_dev_ref_loss: 0.04406	nstep_dev_loss: 0.09204	loop_dev_ref_loss: 0.04332	loop_dev_loss: nan	eltime:  34.41330
epoch: 204	nstep_dev_ref_loss: 0.04479	nstep_dev_loss: 0.09629	loop_dev_ref_loss: 0.04419	loop_dev_loss: nan	eltime:  34.57280
epoch: 205	nstep_dev_ref_loss: 0.04322	nstep_dev_loss: 0.08995	loop_dev_ref_loss: 0.04241	loop_dev_loss: nan	el

epoch: 264	nstep_dev_ref_loss: 0.03053	nstep_dev_loss: 0.07847	loop_dev_ref_loss: 0.02944	loop_dev_loss: nan	eltime:  43.56592
epoch: 265	nstep_dev_ref_loss: 0.03057	nstep_dev_loss: 0.08125	loop_dev_ref_loss: 0.02956	loop_dev_loss: nan	eltime:  43.71520
epoch: 266	nstep_dev_ref_loss: 0.03044	nstep_dev_loss: 0.08161	loop_dev_ref_loss: 0.02944	loop_dev_loss: nan	eltime:  43.86640
epoch: 267	nstep_dev_ref_loss: 0.02997	nstep_dev_loss: 0.07885	loop_dev_ref_loss: 0.02889	loop_dev_loss: nan	eltime:  44.03018
epoch: 268	nstep_dev_ref_loss: 0.02988	nstep_dev_loss: 0.08043	loop_dev_ref_loss: 0.02884	loop_dev_loss: nan	eltime:  44.17952
epoch: 269	nstep_dev_ref_loss: 0.03010	nstep_dev_loss: 0.08323	loop_dev_ref_loss: 0.02913	loop_dev_loss: nan	eltime:  44.32133
epoch: 270	nstep_dev_ref_loss: 0.02958	nstep_dev_loss: 0.08129	loop_dev_ref_loss: 0.02854	loop_dev_loss: nan	eltime:  44.47390
epoch: 271	nstep_dev_ref_loss: 0.02941	nstep_dev_loss: 0.08163	loop_dev_ref_loss: 0.02838	loop_dev_loss: nan	el

In [30]:
# TODO: KeyError: 'loop_dev_loss'

## Model Evaluation
Once training has concluded and the best model has been returned by the trainer, we can call the `evaluate()` method with the best model to evaluate the model's $N$-step and open-loop performance and use the visualizer to generate trace plots for both.

Below, the model's $N$-step traces (top) and open-loop (bottom) traces are shown.

In [32]:
best_outputs = trainer.test(best_model)

/home/tuor369/anaconda3/envs/neuromancer/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/tuor369/anaconda3/envs/neuromancer/lib/python3.8/site-packages/torch/nn/modules/container.py:435: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")


epoch: 299	best_loop_dev_loss: nan	eltime:  279.27509


In [33]:
logger.clean_up()

/home/tuor369/anaconda3/envs/neuromancer/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
